In [17]:
## import libraries

from google.cloud import storage
import gcsfs
import pickle
import pandas as pd
import torch
import copy
from collections import defaultdict
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as multi
import scipy

# create file system to interact with cloud storage
fs = gcsfs.GCSFileSystem()
storage_client = storage.Client()

In [18]:
## pre-defined functions

# load peak list that is the mzlearn run results (use max)
def load_peak_csv(blobs, mzlearn_path):
    # create list of raw data files
    peak_filenames = set()
    for blob in blobs:
        # only select the top folder structure
        # split blobs by '/'
        blob_splited = blob.name.split('/')
        # index 4 is the filename if index 3 is peaks
        if blob_splited[3] == 'peaks':
            peak_filename = blob_splited[4]
            # build the dict
            if peak_filename not in peak_filenames:
                peak_filenames.add(peak_filename)
    # need to be consistent on which file to load
    peak_filenames = list(peak_filenames)
    peak_filenames.sort()
    # print(peak_filenames)
    # if there are 'ax', 'rea' in the file name then load the index 1
    # else load the inde 7
    if peak_filenames[0].split("-")[0] == 'area':
        index_to_load = 7
    else:
        index_to_load = 1
    # create a list of all peaks found
    # read from .csv file on bucket using panda
    fs = gcsfs.GCSFileSystem(project='mzlearn-webapp')
    # convert to list
    peak_filenames = list(peak_filenames)
    peak_filenames.sort()
    # print(peak_filenames)
    # 1 for max 4 for area
    file_name = f'mzlearn-webapp.appspot.com/{mzlearn_path}/peaks/{peak_filenames[index_to_load]}'
    # load peak info
    with fs.open(file_name) as f:
        df = pd.read_csv(f, index_col=False)
    # add code there to detemine coloumns for file names
    return df

# function for hall mark normalization
def hallmark_normalization(peak_table, first_data_col_idx, mzlearn_path, MVT):
    # init file system
    fs = gcsfs.GCSFileSystem()
    storage_client = storage.Client()
    # load data
    data, file_order, bboxes, fingerprint_idxs = load_data(peak_table,
                                                           first_data_col_idx=first_data_col_idx,
                                                           dim_signal=1,
                                                           MVT=MVT)
    # load the dict to all hallmarks path
    dct_hallmarks_name = "pruned_dct_hallmarks.pkl"
    dct_hallmark_path = f'mzlearn-webapp.appspot.com/{mzlearn_path}/maps/'
    with fs.open(dct_hallmark_path + dct_hallmarks_name, "rb") as f:
        dct_hallmarks = pickle.load(f)
    print(f'found {len(dct_hallmarks)} hallmarks')
    # hallmark normalization
    for fn_text, fn in [('sum', lambda arr: np.sum(arr))]:
        for corr_thresh in [0.9]:
            dct_hallmarks_copy = restrain_hallmarks_to_score_above_thresh(
                dct_hallmarks=copy.deepcopy(dct_hallmarks),
                corr_thresh=corr_thresh)
            dct_totals = defaultdict(lambda: 0)
            for f in file_order:
                for idx in dct_hallmarks_copy[f]:
                    dct_totals[f] += fn(dct_hallmarks_copy[f][idx][3][:, 2])
            data_post = data.copy()
            assert data_post.shape[0] == len(list(dct_totals.keys()))
            for i in range(data_post.shape[0]):
                f = file_order[i]
                data_post[i, :] = data_post[i, :] / dct_totals[f]
            data_post[np.isnan(data_post)] = 0
            data_post = np.log2(data_post + 1)
    return data_post

# read peak list data
def load_data(table, first_data_col_idx, dim_signal, MVT=0):

    # only keep non-pool samples i.e. patients
    file_order = [f for f in table.columns[first_data_col_idx:] if not 'pool' in f.lower() and '.mzml' in f.lower()]
    print(f'{len(file_order)} non-pool (patient) samples found')

    data = torch.FloatTensor(table[file_order].astype('float32').to_numpy())
    assert data.shape[0] % dim_signal == 0

    bboxes = [[
        table['rtmin'][i],
        table['mzmin'][i],
        table['rtmax'][i],
        table['mzmax'][i]] for i in range(data.shape[0])]

    if 'mzFingerprint_id' in table.columns:
        fingerprint_idxs = table['mzFingerprint_id']

    # get rid of any feature with MVs
    mask = ((~torch.isnan(data)).sum(dim=-1) / data.shape[-1]) > MVT
    data = data[mask]
    bboxes = [b for i, b in enumerate(bboxes) if mask[i]]
    if 'mzFingerprint_id' in table.columns:
        fingerprint_idxs = [f for i, f in enumerate(fingerprint_idxs) if mask[i]]

    data = data.detach().numpy().transpose()
    print(f'data shape: {data.shape}')
    print(f'{len(bboxes)} bboxes')
    if 'mzFingerprint_id' in table.columns:
        print(f'{len(fingerprint_idxs)} fingerprint idxs')
    else:
        fingerprint_idxs = None
    print('\n')
    return data, file_order, bboxes, fingerprint_idxs

# select only above corr_thresh hallmarks
def restrain_hallmarks_to_score_above_thresh(dct_hallmarks, corr_thresh, list_mzMLs=None):
    if list_mzMLs is None:
        list_mzMLs = list(dct_hallmarks.keys())
    else:
        for f in list_mzMLs: assert f in dct_hallmarks

    hallmark_idxs = list(dct_hallmarks[list_mzMLs[0]].keys())
    num_initial = len(hallmark_idxs)

    new_dct_hallmarks = {f: {} for f in list_mzMLs}

    for hallmark_idx in hallmark_idxs:

        if all([dct_hallmarks[f][hallmark_idx][2] > corr_thresh for f in list_mzMLs]):
            for f in list_mzMLs:
                new_dct_hallmarks[f][hallmark_idx] = dct_hallmarks[f][hallmark_idx]

    found_ids = list(new_dct_hallmarks[list_mzMLs[0]].keys())
    print(f'{len(found_ids)} signals with thresh {corr_thresh}')

    return new_dct_hallmarks

In [19]:
## load data stored on the the cloud storage
## metadata_df is the metadata data frame
## peak_df is the mzlearn output peak list data frame

mzlearn_path = 'mzlearn/NOVO_significant/2022-01-31 23:25:00'
project_code = 'NASH_mouse_pos'

print("load metadata")
metadata_path = f'mzlearn-webapp.appspot.com/{mzlearn_path}/feature_engine/{project_code}/metadata.pkl'
with fs.open(metadata_path, 'rb') as handle:
    metadata_df = pickle.load(handle)

# load found peak list
print("load peaks from mzlearn run")
blobs = storage_client.list_blobs('mzlearn-webapp.appspot.com', prefix=f'{mzlearn_path}/peaks')
peak_df = load_peak_csv(blobs, mzlearn_path)


load metadata
load peaks from mzlearn run


In [20]:
## do hallmark normalization

# hard coded for now the column where file name starts
first_data_col_idx = 10
data_post = hallmark_normalization(peak_df, first_data_col_idx, mzlearn_path, 0)
print(f'normalized data from {len(data_post)} files for {len(data_post[0])} found peaks')
# updata the data frame with normalized intensity values starting from the first_data_col_idx till the end of file
for idx in range(first_data_col_idx, first_data_col_idx + len(data_post)):
    peak_df.iloc[:, idx] = data_post[idx - first_data_col_idx][:]
print("peak_df updated with hallmark normalized Data")

36 non-pool (patient) samples found
data shape: (36, 6775)
6775 bboxes
6775 fingerprint idxs


found 36 hallmarks
269 signals with thresh 0.9
normalized data from 36 files for 6775 found peaks
peak_df updated with hallmark normalized Data


Data normalized, custom data analylsis can be performed to updated peak_df (mzlearn peak result list)

In [24]:
print("test")

import numpy as np
import nbinteract as nbi

def normal(mean, sd):
    '''Returns 1000 points drawn at random fron N(mean, sd)'''
    return np.random.normal(mean, sd, 1000)

normal(10, 1.0)

# Plot aesthetics
options = {
    'xlim': (-2, 12),
    'ylim': (0, 0.7),
    'bins': 20
}

# Pass in the `normal` function and let user change mean and sd.
# Whenever the user interacts with the sliders, the `normal` function
# is called and the returned data are plotted.
nbi.hist(normal, mean=(0, 10), sd=(0, 2.0), options=options)


test
